In [2]:
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, get_sun, get_moon, SkyCoord
import astropy.units as u
import numpy as np

In [3]:
def calculate_intercept(observed_altitude, celestial_body, assumed_position, observation_time):
    """
    Perform a sight reduction to calculate the intercept (distance and direction) and azimuth.
    
    Parameters:
    - observed_altitude: Observed altitude of the celestial body (degrees).
    - celestial_body: Astropy SkyCoord object for the celestial body.
    - assumed_position: EarthLocation object for the assumed observer position.
    - observation_time: Astropy Time object for the observation time.

    Returns:
    - intercept: Distance between observed and calculated altitude (nautical miles).
    - azimuth: Calculated azimuth of the celestial body (degrees).
    """
    # Create an AltAz frame for the assumed position
    altaz_frame = AltAz(location=assumed_position, obstime=observation_time)

    # Transform the celestial body’s coordinates to AltAz
    body_altaz = celestial_body.transform_to(altaz_frame)

    # Extract calculated altitude and azimuth
    calculated_altitude = body_altaz.alt.deg
    azimuth = body_altaz.az.deg

    # Calculate the intercept (difference in altitude)
    intercept = (observed_altitude - calculated_altitude) * 60  # Convert degrees to nautical miles

    return intercept, azimuth


In [4]:
def main():
    # Input data
    observed_altitude = 45.23  # Observed altitude of the Sun (degrees)
    celestial_body_name = "sun"  # Specify 'sun' or 'moon' or use RA/Dec for stars
    assumed_lat = 40.7128  # Assumed latitude (degrees)
    assumed_lon = -74.0060  # Assumed longitude (degrees)
    observation_time = Time.now()  # Observation time in UTC

    # Define the celestial body
    if celestial_body_name.lower() == "sun":
        celestial_body = get_sun(observation_time)
    elif celestial_body_name.lower() == "moon":
        celestial_body = get_moon(observation_time)
    else:
        # Use RA/Dec for stars (example: Polaris)
        celestial_body = SkyCoord(ra=2.530301028*u.hourangle, dec=89.264109444*u.deg)

    # Assumed position of the observer
    assumed_position = EarthLocation(lat=assumed_lat*u.deg, lon=assumed_lon*u.deg, height=0*u.m)

    # Perform sight reduction
    intercept, azimuth = calculate_intercept(observed_altitude, celestial_body, assumed_position, observation_time)

    # Display the results
    print(f"Observed Altitude: {observed_altitude:.2f}°")
    print(f"Intercept (difference in altitude): {abs(intercept):.2f} nautical miles")
    print(f"Direction of intercept: {'Toward' if intercept > 0 else 'Away from'} celestial body")
    print(f"Azimuth of Celestial Body: {azimuth:.2f}°")

In [6]:
def calculate_limb_altitudes(center_altitude, celestial_body, observation_time, observer_location):
    """Calculate lower and upper limb altitudes."""
    # Calculate the angular radius of the celestial body
    if celestial_body == "sun":
        body = get_sun(observation_time)
    elif celestial_body == "moon":
        body = get_moon(observation_time)
    else:
        raise ValueError("Only 'sun' or 'moon' supported for now.")
    
    # AltAz frame for the observer
    altaz_frame = AltAz(location=observer_location, obstime=observation_time)
    body_altaz = body.transform_to(altaz_frame)

    # Angular diameter
    angular_diameter = body.size.to(u.deg)  # Angular diameter in degrees
    angular_radius = angular_diameter / 2

    # Calculate limb altitudes
    lower_limb = center_altitude - angular_radius.value
    upper_limb = center_altitude + angular_radius.value

    return lower_limb, upper_limb

In [5]:
if __name__ == "__main__":
    main()

Observed Altitude: 45.23°
Intercept (difference in altitude): 3910.27 nautical miles
Direction of intercept: Toward celestial body
Azimuth of Celestial Body: 102.77°


In [ ]:
    observed_altitude = 45.23  # Observed altitude of the Sun (degrees)
    celestial_body_name = "sun"  # Specify 'sun' or 'moon' or use RA/Dec for stars
    assumed_lat = 40.7128  # Assumed latitude (degrees)
    assumed_lon = -74.0060  # Assumed longitude (degrees)
    observation_time = Time.now()  # Observation time in UTC